## Imports

In [ ]:
from copy import deepcopy

from pyglotaran_extras import plot_simple_overview
from pyglotaran_extras.compat import convert
from pyglotaran_extras.io import setup_case_study

from glotaran.io import load_dataset, load_parameters, load_scheme, save_dataset

### Load data

In [ ]:
experiment_data = {"dataset1": load_dataset("data/data.ascii")}

### Analysis without equal area penalties

Example of a two step optimization
- First do a (partial) optimization with first scheme
- Extract the optimized schemed and adjust its settings
- Optimize again with the second scheme

In [ ]:
parameters = load_parameters("models/parameters.yml")
scheme = load_scheme("models/scheme.yml")
scheme.load_data(experiment_data)
scheme2 = deepcopy(scheme)

In [ ]:
result_no_penalties_first_run = scheme.optimize(
    parameters=parameters, maximum_number_function_evaluations=5
)
convert(result_no_penalties_first_run)

In [ ]:
# optimization with second scheme,
# where we change the maximum number of function evaluations
result_no_penalties = scheme.optimize(
    parameters=result_no_penalties_first_run.parameters_optimized,
    maximum_number_function_evaluations=3,
)
convert(result_no_penalties)

#### Saving results and creating plots

In [ ]:
results_folder, _ = setup_case_study(output_folder_name="pyglotaran_examples_results_staging")
# save the first run:
result_no_penalties_first_run.save(results_folder / "no_penalties_first_run", allow_overwrite=True)
# also save the second run:
result_no_penalties.save(results_folder / "no_penalties", allow_overwrite=True)
convert(result_no_penalties)

In [ ]:
plot_simple_overview(convert(result_no_penalties_first_run.data["dataset1"]))
plot_simple_overview(convert(result_no_penalties.data["dataset1"]));

### Analysis using equal area penalties

This time we use equal parameter in our model, allowing us to estimate the relative amplitude between the two species.

In [ ]:
parameters_equal_area = load_parameters("models/parameters_equal_area_penalties.yml")

scheme_equal_area = load_scheme("models/scheme_equal_area_penalties.yml")
scheme_equal_area.load_data(experiment_data)

result_equal_area_first_run = scheme_equal_area.optimize(parameters=parameters_equal_area)
convert(result_equal_area_first_run)

In [ ]:
result_equal_area_first_run.save(results_folder / "with_penalties_first_run", allow_overwrite=True)

In [ ]:
result_equal_area = scheme_equal_area.optimize(
    parameters=result_equal_area_first_run.parameters_optimized
)
result_equal_area.save(results_folder / "with_penalties", allow_overwrite=True)
convert(result_equal_area)

In [ ]:
plot_simple_overview(convert(result_equal_area_first_run.data["dataset1"]))
plot_simple_overview(convert(result_equal_area.data["dataset1"]));

In [ ]:
for data_path in results_folder.rglob("*.nc"):
    save_dataset(
        convert(load_dataset(data_path)),
        data_path.parent.parent / data_path.name,
        allow_overwrite=True,
    )
    data_path.unlink()

## Scheme generation code

In [ ]:
from glotaran.io import load_parameters
from glotaran.utils.json_schema import create_model_scheme_json_schema

parameters = load_parameters("models/parameters_equal_area_penalties.yml")
create_model_scheme_json_schema("models/schema.json", parameters)